### Тема: Аутентификация и авторизация в REST API

1. **Цель урока**:
   - Изучить способы защиты REST API, используя JWT (JSON Web Tokens).
   - Научиться реализовывать проверку прав доступа и защищать маршруты.

2. **План урока**:
   1. **Введение в аутентификацию и авторизацию**:
      - Разница между аутентификацией (подтверждение личности пользователя) и авторизацией (проверка прав доступа).
      - Использование токенов JWT для обеспечения безопасности API.

   2. **Установка необходимых пакетов**:
      - Установим библиотеку для работы с JWT:
        ```bash
        go get -u github.com/dgrijalva/jwt-go
        ```

   3. **Создание простого механизма регистрации и аутентификации**:
      - Реализуем маршруты для регистрации (`/register`) и входа в систему (`/login`), чтобы генерировать JWT.

   4. **Добавление middleware для защиты маршрутов**:
      - Создадим middleware для проверки JWT-токена, который будет использоваться для защиты маршрутов, например `/books`.

   5. **Пример кода**:

      - Добавим функцию генерации JWT-токена:

        ```go
        package main

        import (
            "time"
            "github.com/gin-gonic/gin"
            "github.com/dgrijalva/jwt-go"
        )

        var jwtKey = []byte("my_secret_key")

        type Credentials struct {
            Username string `json:"username"`
            Password string `json:"password"`
        }

        type Claims struct {
            Username string `json:"username"`
            jwt.StandardClaims
        }

        func generateToken(username string) (string, error) {
            expirationTime := time.Now().Add(5 * time.Minute)
            claims := &Claims{
                Username: username,
                StandardClaims: jwt.StandardClaims{
                    ExpiresAt: expirationTime.Unix(),
                },
            }
            token := jwt.NewWithClaims(jwt.SigningMethodHS256, claims)
            return token.SignedString(jwtKey)
        }
        ```

      - Реализуем маршруты `/login` и middleware для проверки JWT:

        ```go
        func login(c *gin.Context) {
            var creds Credentials
            if err := c.BindJSON(&creds); err != nil {
                c.JSON(http.StatusBadRequest, gin.H{"message": "invalid request"})
                return
            }

            // Здесь добавим простую проверку пароля
            if creds.Username != "user" || creds.Password != "password" {
                c.JSON(http.StatusUnauthorized, gin.H{"message": "unauthorized"})
                return
            }

            token, err := generateToken(creds.Username)
            if err != nil {
                c.JSON(http.StatusInternalServerError, gin.H{"message": "could not create token"})
                return
            }

            c.JSON(http.StatusOK, gin.H{"token": token})
        }

        func authMiddleware() gin.HandlerFunc {
            return func(c *gin.Context) {
                tokenString := c.GetHeader("Authorization")

                claims := &Claims{}
                token, err := jwt.ParseWithClaims(tokenString, claims, func(token *jwt.Token) (interface{}, error) {
                    return jwtKey, nil
                })

                if err != nil || !token.Valid {
                    c.JSON(http.StatusUnauthorized, gin.H{"message": "unauthorized"})
                    c.Abort()
                    return
                }

                c.Next()
            }
        }
        ```

      - Применим middleware к защищенным маршрутам:

        ```go
        func main() {
            router := gin.Default()

            router.POST("/login", login)

            protected := router.Group("/")
            protected.Use(authMiddleware())
            {
                protected.GET("/books", getBooks)
                protected.POST("/books", createBook)
                // другие защищенные маршруты
            }

            router.Run(":8080")
        }
        ```

3. **Практическая часть**:
   - Студенты реализуют регистрацию и проверку пользователей, используя заранее созданный список пользователей.
   - Добавление проверки ролей, например, для доступа к определенным маршрутам.

4. **Задание на дом**:
   - Добавить обработку истечения срока действия токенов.
   - Реализовать рефреш токенов для продления срока действия аутентификации.

Этот урок позволит студентам получить представление о безопасности в веб-приложениях и реализовать основные механизмы защиты API на практике.